# Quickstart

In this quick tutorial, we'll show the basic ideas on what you can do with `zfit`, without going into much detail or performing advanced tasks.


In [ ]:
import matplotlib.pyplot as plt
import mplhep
import numpy as np
import zfit
import zfit.z.numpy as znp  # numpy-like backend

## Create observables

The observable space in which PDFs are defined is created with the `Space` class

In [ ]:
obs = zfit.Space('x', -10, 10)

## Create data

We create some unbinned data using `numpy`. Other constructors, e.g. for `ROOT` files are also available.

In [ ]:
mu_true = 0
sigma_true = 1

data_np = np.random.normal(mu_true, sigma_true, size=10000)
data = zfit.Data(data=data_np, obs=obs)

## Create a PDF to fit

Let's create a Gaussian PDF so we can fit the dataset. To do this, first we create the fit parameters, which follow a convention similar to `RooFit`:

```
zfit.Parameter(name, initial_value, lower_limit (optional), upper_limit (optional), other options)
```

In [ ]:
mu = zfit.Parameter("mu", 2.4, -1., 5., step_size=0.001)  # step_size is not mandatory but can be helpful
sigma = zfit.Parameter("sigma", 1.3, 0, 5., step_size=0.001)  # it should be around the estimated uncertainty

Now we instantiate a Gaussian from the zfit PDF library (more on how to create your own PDFs later)

In [ ]:
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma)

In [ ]:
gauss.plot.plotpdf()

This pdf contains several useful methods, such as calculating a probability, calculating its integral, sampling etc.

In [ ]:
# Let's get some probabilities.
consts = [-1, 0, 1]
probs = gauss.pdf(consts)
print(f"x values: {consts}\nresult:   {probs}")

## Fitting

To fit, we need to take three steps: create the negative $\log\mathcal{L}$, instantiate a minimizer and then minimize the likelihood.

In [ ]:
# Create the negative log likelihood

nll = zfit.loss.UnbinnedNLL(model=gauss, data=data)  # loss

# Load and instantiate a minimizer
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(loss=nll)

print(result)

And we can plot the result to see how it went.

In [ ]:
%matplotlib inline

In [ ]:
n_bins = 50
mplhep.histplot(data.to_binned(50))
rescale = obs.v1.volume / n_bins * float(data.nevents)
ax = gauss.plot.plotpdf(scale=rescale)

# x = np.linspace(*obs.v1.limits, num=1000)
# probs = gauss.pdf(x)
# _ = plt.plot(x, rescale * probs)

In [ ]:
obs.v1.volume

The `FitResult` that we obtained contains information about the minimization and can now be used to calculate the errors

In [ ]:
print(f"Function result: {result.fmin}", result.fmin)
print(f"Converged: {result.converged} and valid: {result.valid}", )
print(result)

In [ ]:
# we still have access to everything
result.loss.model[0]

In [ ]:
hesse_errors = result.hesse()
minos_errors = result.errors()

In [ ]:
print(result)

## Storing the result

Everything is accessible, feel free to store it in your own format


In [ ]:
dumped = zfit.dill.dumps(result)  # like pickle
loaded = zfit.dill.loads(dumped)
loadedpdf = loaded.loss.model[0]
loadedpdf.plot.plotpdf()

In [ ]:
zfit.hs3.dumps(nll)  # experimental, human-readable serialization